---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [32]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [33]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import numpy as np
    import pandas as pd
    temp = (pd.read_table('university_towns.txt', header=None).rename(columns={0: 'States'}))
    #return temp
    university_towns = pd.DataFrame(columns=['State','RegionName'])
    for states1 in temp['States']:
        if '[edit]' in states1:
            st = states1.split('[')[0]
            continue
        temp1 = pd.DataFrame([[st, states1.split(' (')[0]]],columns=["State","RegionName"])       
        university_towns = university_towns.append(temp1, ignore_index=True)
    return university_towns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [34]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    import pandas as pd
    temp = pd.read_excel('gdplev.xls', skiprows = 7)
    temp = pd.DataFrame(temp[['Unnamed: 4','Unnamed: 5']]).rename(columns ={'Unnamed: 4': 'YearQtr', 'Unnamed: 5': 'GDP'})    
    temp = temp[temp['YearQtr'].apply(lambda x: int(x.split('q')[0])) >= 2000].reset_index()[['YearQtr', 'GDP']]    
    for i in range(len(temp)-2):
        if ((temp.iloc[i+1]['GDP'] < temp.iloc[i]['GDP']) & (temp.iloc[i+2]['GDP'] < temp.iloc[i+1]['GDP'])):
            recession_start = temp.iloc[i]['YearQtr']
            break
    return recession_start
get_recession_start()

'2008q3'

In [35]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    import pandas as pd
    temp = pd.read_excel('gdplev.xls', skiprows = 7)
    temp = pd.DataFrame(temp[['Unnamed: 4','Unnamed: 5']]).rename(columns ={'Unnamed: 4': 'YearQtr', 'Unnamed: 5': 'GDP'}) 
    temp['YQ'] = temp['YearQtr'].apply(lambda x: x.split('q')[0]) + temp['YearQtr'].apply(lambda x: x.split('q')[1])
    temp['YQ'] = temp['YQ'].apply(lambda x: int(x))
    recession_start = int(get_recession_start().split('q')[0]+get_recession_start().split('q')[1])
    temp = temp[temp['YQ'] > recession_start]
    #temp = temp[temp['YearQtr'].apply(lambda x: int(x.split('q')[0])) >= int(get_recession_start()[0].split('q')[0])].reset_index()[['YearQtr', 'GDP']]     
    for i in range(2,len(temp)):
        if ((temp.iloc[i-1]['GDP'] < temp.iloc[i]['GDP']) & (temp.iloc[i-2]['GDP'] < temp.iloc[i-1]['GDP'])):
            recession_end=(temp.iloc[i]['YearQtr'])
            break
    return recession_end
get_recession_end()

'2009q4'

In [36]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    import pandas as pd
    temp = pd.read_excel('gdplev.xls', skiprows = 7)
    temp = pd.DataFrame(temp[['Unnamed: 4','Unnamed: 5']]).rename(columns ={'Unnamed: 4': 'YearQtr', 'Unnamed: 5': 'GDP'}) 
    temp['YQ'] = temp['YearQtr'].apply(lambda x: x.split('q')[0]) + temp['YearQtr'].apply(lambda x: x.split('q')[1])
    temp['YQ'] = temp['YQ'].apply(lambda x: int(x))
    recession_start = int(get_recession_start().split('q')[0]+get_recession_start().split('q')[1])
    recession_end = int(get_recession_end().split('q')[0]+get_recession_end().split('q')[1])
    temp = temp[(temp['YQ']>recession_start) & (temp['YQ']<recession_end)].set_index('YearQtr')
    return np.argmin(temp['GDP'])
get_recession_bottom()

'2009q2'

In [37]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import numpy as np
    import pandas as pd
    temp = pd.read_csv('City_Zhvi_AllHomes.csv')
    temp = temp.fillna(0)
    temp['State'] = temp['State'].apply(lambda x: states[x])
    temp = temp.reset_index().set_index(['State','RegionName'])
    housing_data = pd.DataFrame()
    for i in range(2000,2016):
        housing_data[(str(i)+'q1')] = temp[str(i)+'-'+'01'] + temp[str(i)+'-'+'02'] + temp[str(i)+'-'+'03']
        housing_data[(str(i)+'q2')] = temp[str(i)+'-'+'04'] + temp[str(i)+'-'+'05'] + temp[str(i)+'-'+'06']
        housing_data[(str(i)+'q3')] = temp[str(i)+'-'+'07'] + temp[str(i)+'-'+'08'] + temp[str(i)+'-'+'09']
        housing_data[(str(i)+'q4')] = temp[str(i)+'-'+'10'] + temp[str(i)+'-'+'11'] + temp[str(i)+'-'+'12']
    i = 2016
    housing_data[(str(i)+'q1')] = temp[str(i)+'-'+'01'] + temp[str(i)+'-'+'02'] + temp[str(i)+'-'+'03']
    housing_data[(str(i)+'q2')] = temp[str(i)+'-'+'04'] + temp[str(i)+'-'+'05'] + temp[str(i)+'-'+'06']
    housing_data[(str(i)+'q3')] = temp[str(i)+'-'+'07'] + temp[str(i)+'-'+'08']
    return housing_data    
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1546400,1568400,1584200,1596800,1622400,1671600,1718500,1748600,1774900,1174400
California,Los Angeles,621200.0,643400.0,662900.0,678500.0,699000.0,717300.0,735200.0,759100.0,785900.0,818100.0,...,1494100,1527200,1556600,1586400,1614500,1641800,1673200,1698100,1732400,1168100
Illinois,Chicago,415200.0,430900.0,443600.0,456400.0,470800.0,485400.0,499200.0,511300.0,526500.0,532700.0,...,577900,587300,603800,603200,618100,624900,623700,618200,624600,424000
Pennsylvania,Philadelphia,159000.0,160900.0,162400.0,164100.0,166000.0,166600.0,168800.0,172600.0,177400.0,182200.0,...,341200,345900,347000,348600,353900,363700,366600,370300,380800,257400
Arizona,Phoenix,335500.0,343100.0,348000.0,352200.0,358800.0,364700.0,368100.0,372900.0,379600.0,385100.0,...,492800,496100,505500,514600,522500,537200,551500,563700,574300,390400
Nevada,Las Vegas,397800.0,403100.0,406200.0,411000.0,418600.0,425200.0,430100.0,438400.0,448000.0,452800.0,...,510200,520200,526400,532500,544800,560300,571900,583800,591600,399900
California,San Diego,668700.0,703100.0,736300.0,768100.0,801600.0,828800.0,853500.0,875800.0,903700.0,938600.0,...,1440600,1467100,1489300,1510100,1536200,1559300,1576400,1588000,1608700,1079500
Texas,Dallas,253400.0,251600.0,254600.0,263500.0,269200.0,267900.0,267200.0,272700.0,277700.0,281400.0,...,319900,326700,334600,341100,363400,385700,403800,421500,433800,298600
California,San Jose,1122800.0,1219700.0,1295600.0,1366500.0,1412000.0,1410600.0,1370400.0,1336700.0,1324300.0,1373300.0,...,2038200,2091100,2144800,2194300,2270200,2329200,2367400,2410800,2456800,1644400


In [38]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()
    univ = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    housing['ratio'] = (housing[bottom] - housing[start])/housing[bottom]
    housing = housing[[bottom,start,'ratio']].reset_index()
    ut = pd.DataFrame(univ.merge(housing, how='inner', right_on = ['State','RegionName'], left_on = ['State', 'RegionName']))
    nut = pd.DataFrame(univ.merge(housing, how ='outer', right_on=['State','RegionName'], left_on = ['State', 'RegionName']))
    nut['CT'] = nut['RegionName'].apply(lambda x: x in set(univ['RegionName']))
    nut = nut[nut['CT'] == False]
    p_val = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    if p_val.pvalue <  0.05:
        different = True
    else:
        different = False
    if ut['ratio'].dropna().mean() > nut['ratio'].dropna().mean():
        better = 'university town'
    else:
        better = 'non university town'
    ans = (different, p_val.pvalue, better)
    return ans
run_ttest()

(True, 0.033097881884895035, 'university town')